In [ ]:
using Revise, Genie, DelimitedFiles, DCAUtils, JLD2, PyPlot, KitMSA, Statistics

@load "../data_Genie/pars_beta_nc.jld2"


beta_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 0.9);


#define paths
path_wt_tem = "../Gen.jl/data/alignments/refs/TEM1_pfam_DNA.fasta";
path_wt_pse = "../Gen.jl/data/alignments/refs/PSE1_pfam_DNA.fasta";

#load wt
wt_tem = join(readdlm(path_wt_tem, skipstart = 1));
wt_pse = join(readdlm(path_wt_pse, skipstart = 1));

#transform wt_dna in amino
L_tem = Int64(length(wt_tem)/3);
tem_DNA_seq = [wt_tem[((i-1)*3 +1):(i*3)] for i in 1:L_tem];
tem = Int.([Genie.cod2amino[codon] for codon in tem_DNA_seq]);

L_pse = Int64(length(wt_pse)/3);
pse_DNA_seq = [wt_pse[((i-1)*3 +1):(i*3)] for i in 1:L_pse];
pse = Int.([Genie.cod2amino[codon] for codon in pse_DNA_seq]);


N_start = 10000; @time m = Genie.run_dir_evol(pse[3:end-1], N_start, h[:,3:end-1], J[:,3:end-1,:,3:end-1];
                   rounds = 4,
                   seq_reads = 1000,
                   each_step = 1,
                   temp = 1.,  
                   mu = 10^-2, mu_bind = 18.6);


mean(ham_dist(pse[3:end-1], m.step_msa[end]))


N_start = 10000; @time m_n = Genie.run_dir_evol_nucleo(pse_DNA_seq[3:end-1], N_start, h[:,3:end-1], J[:,3:end-1,:,3:end-1];
                   rounds = 4,
                   seq_reads = 1000,
                   each_step = 1,
                   temp = 1.,  
                   mu = 10^-2, mu_bind = 18.6);


mean(ham_dist(pse[3:end-1], m_n.step_msa[end]))


function give_pos(idx)
    seq_site = div(idx,3) + 1
    nucleo_site = mod(idx,3)
            if seq_site == 0
                seq_site = 1
            end
            if nucleo_site == 0
                nucleo_site = 3
            end
    return seq_site, nucleo_site
end

start_msa = hcat([beta_msa[:,1] for i in 1:10000]...); @time res = run_evolution(start_msa, 
    h, 
    J,
    p = 0.5, 
    temp = 1., 
    N_steps = 5*10^2,  
    N_points = 20, 
    verbose = true);




M = 1000000; temp = 1; mu_bind = 18.6; mu = 10^-1;  start_msa = Int.(hcat([pse for i in 1:M]...)); L = size(start_msa,1 );msa = deepcopy(
    start_msa); 
@time Genie.random_mutations!(msa, mu);
#@time abund_dict = Genie.seq_count_dict(msa);
@time Genie.selection!(msa, h, J, temp = 1., mu_bind = 18.6);
final_msa = zeros(Int, L, M); @time Genie.amplification!(final_msa, msa);

mean(ham_dist(Int.(pse), final_msa))



M = 10000; temp = 1.; mu_bind = 18.6; mu = 10^-1;  start_msa = Int.(hcat([pse for i in 1:M]...)); L = size(start_msa,1);msa = deepcopy(
    start_msa); 
@btime Genie.random_mutations!($(msa), $(mu));
@btime abund_dict = Genie.seq_count_dict($(msa));
@btime Genie.selection!($(msa), $(abund_dict), $(h), $(J), temp = temp, mu_bind = mu_bind);
final_msa = zeros(Int, L, M); @btime Genie.amplification!($(final_msa), $(msa), $(abund_dict));










check_equilibration("../figures/beta/", beta_msa, res.step_msa, res.steps)

Genie.check_timescales(res.step_msa, 
    beta_msa, h, J, res.steps, "../figures/beta/dynamics.png");
